In [1]:
from keras.datasets import imdb

NUM_WORDS = 10000

(dev_X, dev_y) , (test_X, test_y) = imdb.load_data(num_words=NUM_WORDS,
                                                      skip_top=0,
                                                      maxlen=None)

Using TensorFlow backend.


The point of all this thing is to try and beat their score
http://nbviewer.jupyter.org/github/fchollet/deep-learning-with-python-notebooks/blob/master/3.5-classifying-movie-reviews.ipynb

In [2]:
dev_X.shape

(25000,)

In [3]:
dev_y.shape

(25000,)

In [4]:
import numpy as np

In [5]:
from keras.layers import Dense, Dropout, Embedding, LSTM
from keras.models import Sequential
from keras.callbacks import TensorBoard

First I will try sequental model with few dense layers without autoencoder

In [6]:
def vectorize_y(Y):
    return np.asarray(Y).astype('float32')

In [15]:
y = vectorize_y(dev_y)
y[:2]

array([ 1.,  0.], dtype=float32)

In [7]:
def one_hot_encode(X):
    seq = np.zeros((len(X),NUM_WORDS))
    for (i,val) in enumerate(X):
        seq[i][val] = 1
    return seq

In [8]:
X = one_hot_encode(dev_X)
X[:2]

array([[ 0.,  1.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  0.,  0.,  0.]])

In [120]:
model = Sequential([
    Dense(16, input_shape=(NUM_WORDS,), activation="tanh"),
    Dropout(0.5),
    Dense(64, activation="tanh"),
    Dense(1, activation="sigmoid")
])

In [121]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [122]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 16)                160016    
_________________________________________________________________
dropout_22 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_71 (Dense)             (None, 64)                1088      
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 65        
Total params: 161,169
Trainable params: 161,169
Non-trainable params: 0
_________________________________________________________________


In [123]:
model.fit(X, y, epochs=3, validation_split=0.2, batch_size=712, callbacks=[TensorBoard(log_dir="./logs")])

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 1s 45us/step - loss: 0.5426 - acc: 0.7641 - val_loss: 0.3646 - val_acc: 0.8626
Epoch 2/3
20000/20000 [==============================] - 1s 42us/step - loss: 0.2909 - acc: 0.8914 - val_loss: 0.2804 - val_acc: 0.8886
Epoch 3/3
20000/20000 [==============================] - 1s 42us/step - loss: 0.2105 - acc: 0.9242 - val_loss: 0.2724 - val_acc: 0.8942


In [24]:
X_test = one_hot_encode(test_X)
y_test = vectorize_y(test_y)

In [124]:
results = model.evaluate(X_test, y_test)
results

25000/25000 [==============================] - 2s 62us/step


[0.2873726367521286, 0.88404000000000005]

Yup it seems I have won this one [0.2873726367521286, 0.88404000000000005] :)

And now last but not least, lets check how simple long short term memory network handles the classification

In [9]:
def pad_seq(X):
    max_len_seq = max([len(x) for x in X])
    seq = []
    for x in X:
        x_clone = np.append(x, [np.zeros((max_len_seq - len(x)), dtype="int32") ])
        seq.append(x_clone)
    return (max_len_seq, np.array(seq))

In [10]:
max_len_seq, X = pad_seq(dev_X)
X[:2]

array([[   1,   14,   22, ...,    0,    0,    0],
       [   1,  194, 1153, ...,    0,    0,    0]])

In [11]:
model = Sequential([
    Embedding(NUM_WORDS, 128),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation="sigmoid")
])

In [12]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs=12, validation_split=0.2, batch_size=32)

Seems a lstm with a 2,6k as input is really slow to train so I will skip it and try something else

In [18]:
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(dev_X, maxlen=100)

In [30]:
model = Sequential([
    Embedding(NUM_WORDS, 128),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation="sigmoid")
])

In [31]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 128)         1280000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(X, y, epochs=3, validation_split=0.2, batch_size=32)

Train on 20000 samples, validate on 5000 samples
Epoch 1/3
20000/20000 [==============================] - 72s 4ms/step - loss: 0.4806 - acc: 0.7672 - val_loss: 0.4187 - val_acc: 0.8112
Epoch 2/3
20000/20000 [==============================] - 70s 4ms/step - loss: 0.3234 - acc: 0.8674 - val_loss: 0.3910 - val_acc: 0.8210
Epoch 3/3
20000/20000 [==============================] - 69s 3ms/step - loss: 0.2524 - acc: 0.8999 - val_loss: 0.3794 - val_acc: 0.8352


In [34]:
X_test = pad_sequences(test_X, maxlen=100)

In [37]:
results = model.evaluate(X_test, y_test, batch_size=512)
results

25000/25000 [==============================] - 1s 50us/step


[0.38172442971229553, 0.83604000009536739]